# Importando Bibliotecas

In [1]:
import sqlite3

import pandas as pd

import plotly.graph_objects as go

from statsmodels.tsa.seasonal import seasonal_decompose 

import warnings
warnings.filterwarnings('ignore')

# Funções

In [2]:
def get_month(mes):
    if mes == 'Jan':
        return '01'
    elif mes == 'Fev':
        return '02'
    elif mes == 'Mar':
        return '03'
    elif mes == 'Abr':
        return '04'
    elif mes == 'Mai':
        return '05'
    elif mes == 'Jun':
        return '06'
    elif mes == 'Jul':
        return '07'
    elif mes == 'Ago':
        return '08'
    elif mes == 'Set':
        return '09'
    elif mes == 'Out':
        return '10'
    elif mes == 'Nov':
        return '11'
    else:
        return '12'
def na_lista(item, lista):
    if item in lista:
        return item
    else:
        return 'Outra'
def trocar_valor(item, valor_antigo, novo_valor):
    if item == valor_antigo:
        return novo_valor
    else:
        return item

# Conexão Banco de Dados

In [3]:
db = sqlite3.connect('../BD/projeto_integrador.db')

# Lendo os Dados

In [4]:
query = '''
SELECT * 
FROM VENDAS
'''

df = pd.read_sql_query(query, db)
df

,user_id,city,country,device,os,platform,device_type,gp:Universidade,gp:Tipo de Universidade,MesAno
0,124557,São Luís,Brazil,Windows,Chrome 101,Web,Windows,UFMA (Universidade Federal do Maranhão),Pública,Fev22
1,41262,Florianópolis,Brazil,Linux,Chrome 96,Web,Linux,UFSC (Universidade Federal de Santa Catarina),Pública,Fev22
2,98102,Palmas,Brazil,Windows,Edge 100,Web,Windows,UFT (Universidade Federal do Tocantins),Pública,Fev22
3,138808,Teresina,Brazil,Android,Chrome Mobile 78,Web,Android,UFPI (Universidade Federal do Piauí),Pública,Fev22
4,144725,São Paulo,Brazil,Apple iPhone,Mobile Safari 15,Web,Apple iPhone,UNIVATES (Centro Universitário UNIVATES),Privada,Fev22
...,...,...,...,...,...,...,...,...,...,...
374165,393427,Salvador,Brazil,Apple iPhone,ios 13.1,iOS,Apple iPhone X,UFBA (Universidade Federal da Bahia),,Jul18
374166,394749,São Paulo,Brazil,Android,Chrome 67,Web,Android,UFABC (Universidade Federal do ABC),Pública,Jul18
374167,394930,Pelotas,Brazil,Windows,Chrome 89,Web,Windows,UFPEL (Universidade Federal de Pelotas),Pública,Jul18
374168,395209,São Bernardo do Campo,Brazil,Windows,Chrome 76,Web,Windows,UFABC (Universidade Federal do ABC),,Jul18


## Campo de Data

In [5]:
df['ano'] = '20'+df['MesAno'].str[-2:]
df['mes'] = df['MesAno'].str[:3]
df['mes'] = df['mes'].map(lambda x: get_month(x))
df['data'] = pd.to_datetime(df['ano']+'-'+df['mes']+'-01')
df

,user_id,city,country,device,os,platform,device_type,gp:Universidade,gp:Tipo de Universidade,MesAno,ano,mes,data
0,124557,São Luís,Brazil,Windows,Chrome 101,Web,Windows,UFMA (Universidade Federal do Maranhão),Pública,Fev22,2022,02,2022-02-01
1,41262,Florianópolis,Brazil,Linux,Chrome 96,Web,Linux,UFSC (Universidade Federal de Santa Catarina),Pública,Fev22,2022,02,2022-02-01
2,98102,Palmas,Brazil,Windows,Edge 100,Web,Windows,UFT (Universidade Federal do Tocantins),Pública,Fev22,2022,02,2022-02-01
3,138808,Teresina,Brazil,Android,Chrome Mobile 78,Web,Android,UFPI (Universidade Federal do Piauí),Pública,Fev22,2022,02,2022-02-01
4,144725,São Paulo,Brazil,Apple iPhone,Mobile Safari 15,Web,Apple iPhone,UNIVATES (Centro Universitário UNIVATES),Privada,Fev22,2022,02,2022-02-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...
374165,393427,Salvador,Brazil,Apple iPhone,ios 13.1,iOS,Apple iPhone X,UFBA (Universidade Federal da Bahia),,Jul18,2018,07,2018-07-01
374166,394749,São Paulo,Brazil,Android,Chrome 67,Web,Android,UFABC (Universidade Federal do ABC),Pública,Jul18,2018,07,2018-07-01
374167,394930,Pelotas,Brazil,Windows,Chrome 89,Web,Windows,UFPEL (Universidade Federal de Pelotas),Pública,Jul18,2018,07,2018-07-01
374168,395209,São Bernardo do Campo,Brazil,Windows,Chrome 76,Web,Windows,UFABC (Universidade Federal do ABC),,Jul18,2018,07,2018-07-01


In [6]:
df.drop(['MesAno', 'ano', 'mes'], axis=1, inplace=True)

# EDA

## Dicionário de Dados

- ```user_id``` - ID do usuário
- ```city``` - Cidade do usuário
- ```country``` - País 
- ```device``` - Dispositivo (Windows, Android, iPhone, etc)
- ```os``` - Sistema Operacional (ou navegador web)
- ```platform``` - Plataforma (Web, Android, iOS ou Vazio)
- ```device_type``` - Tipo de dispositivo
- ```gp:Universidade``` - Universidade do usuário
- ```gp:Tipo de Universidade``` - Tipo de universidade do usuário
- ```MesAno``` - Mes e Ano da Venda

### Vendas por Mês

In [7]:
vendas = pd.pivot_table(df, values='user_id', aggfunc='count', columns='data').T
vendas.reset_index(inplace=True)
vendas

,data,user_id
0,2018-01-01,1819
1,2018-02-01,2012
2,2018-03-01,4035
3,2018-04-01,4305
4,2018-05-01,3956
5,2018-06-01,3962
6,2018-07-01,2875
7,2018-08-01,3959
8,2018-09-01,4566
9,2018-10-01,5079


In [8]:
fig = go.Figure(data=go.Scatter(x=vendas['data'], y=vendas['user_id'], marker={'color': 'gray'}))
fig.update_layout(font=dict(color='black'), title_text='Vendas por mês')
fig.show()

### User ID

In [9]:
df_duplicated = pd.DataFrame(df.duplicated('user_id'), columns=['Duplicated'])
df_duplicated

,Duplicated
0,False
1,False
2,False
3,False
4,False
...,...
374165,True
374166,True
374167,False
374168,True


In [10]:
fig = go.Figure(data=go.Bar(x=['Duplicados', 'Únicos'], y=df_duplicated['Duplicated'].value_counts(), marker={'color': 'gray'}))
fig.update_layout(font=dict(color='black'), title_text='Contagem de usuários duplicados', barmode='group')
fig.show()

##### Usuário com maior número de compras

In [11]:
user = df['user_id'].value_counts().sort_values(ascending=False).index[0]
maior_compra = df['user_id'].value_counts().sort_values(ascending=False)[0]

print(f'O usuário que mais comprou foi de id: {user} e ele comprou {maior_compra} vezes')

O usuário que mais comprou foi de id: 211842 e ele comprou 51 vezes


### City

In [12]:
df_cidades = pd.DataFrame(df['city'].value_counts().sort_values(ascending=False)[:10])
df_cidades.reset_index(inplace=True)
df_cidades.columns = ['Cidade', 'Vendas']
df_cidades

,Cidade,Vendas
0,Rio de Janeiro,60018
1,São Paulo,26509
2,Belo Horizonte,13872
3,Campinas,11633
4,Curitiba,9954
5,Recife,8720
6,Brasília,8583
7,Niterói,7616
8,Salvador,6593
9,Fortaleza,5962


In [13]:
fig = go.Figure(data=go.Bar(x=df_cidades['Cidade'], y=df_cidades['Vendas'], marker={'color': 'gray'}))
fig.update_layout(font=dict(color='black'), title_text='Vendas por cidade', barmode='group')
fig.show()

### Country

In [14]:
df_pais = pd.DataFrame(df['country'].value_counts().sort_values(ascending=False)[:3])
df_pais.reset_index(inplace=True)
df_pais.columns = ['País', 'Vendas']
df_pais

,País,Vendas
0,Brazil,368687
1,Portugal,1522
2,United States,1002


In [15]:
fig = go.Figure(data=go.Bar(x=df_pais['País'], y=df_pais['Vendas'], marker={'color': 'gray'}))
fig.update_layout(font=dict(color='black'), title_text='Vendas por país', barmode='group')
fig.show()

### Device

In [16]:
df_device = pd.DataFrame(df['device'].value_counts().sort_values(ascending=False)[:5])
df_device.reset_index(inplace=True)
df_device.columns = ['Device', 'Vendas']
df_device

,Device,Vendas
0,Windows,238800
1,Android,56941
2,Apple iPhone,31093
3,Mac,11270
4,Linux,7661


In [17]:
fig = go.Figure(data=go.Bar(x=df_device['Device'], y=df_device['Vendas'], marker={'color': 'gray'}))
fig.update_layout(font=dict(color='black'), title_text='Vendas por Dispositivo', barmode='group')
fig.show()

### Platform

In [18]:
df_platform = pd.DataFrame(df['platform'].value_counts().sort_values(ascending=False)[:3])
df_platform.reset_index(inplace=True)
df_platform.columns = ['Plataforma', 'Vendas']
df_platform

,Plataforma,Vendas
0,Web,338449
1,Android,24102
2,iOS,10469


In [19]:
fig = go.Figure(data=go.Bar(x=df_platform['Plataforma'], y=df_platform['Vendas'], marker={'color': 'gray'}))
fig.update_layout(font=dict(color='black'), title_text='Vendas por Plataforma', barmode='group')
fig.show()

### Universidade

In [20]:
df['Universidade'] = df['gp:Universidade'].map(lambda x: x.split(' ')[0])
df

,user_id,city,country,device,os,platform,device_type,gp:Universidade,gp:Tipo de Universidade,data,Universidade
0,124557,São Luís,Brazil,Windows,Chrome 101,Web,Windows,UFMA (Universidade Federal do Maranhão),Pública,2022-02-01,UFMA
1,41262,Florianópolis,Brazil,Linux,Chrome 96,Web,Linux,UFSC (Universidade Federal de Santa Catarina),Pública,2022-02-01,UFSC
2,98102,Palmas,Brazil,Windows,Edge 100,Web,Windows,UFT (Universidade Federal do Tocantins),Pública,2022-02-01,UFT
3,138808,Teresina,Brazil,Android,Chrome Mobile 78,Web,Android,UFPI (Universidade Federal do Piauí),Pública,2022-02-01,UFPI
4,144725,São Paulo,Brazil,Apple iPhone,Mobile Safari 15,Web,Apple iPhone,UNIVATES (Centro Universitário UNIVATES),Privada,2022-02-01,UNIVATES
...,...,...,...,...,...,...,...,...,...,...,...
374165,393427,Salvador,Brazil,Apple iPhone,ios 13.1,iOS,Apple iPhone X,UFBA (Universidade Federal da Bahia),,2018-07-01,UFBA
374166,394749,São Paulo,Brazil,Android,Chrome 67,Web,Android,UFABC (Universidade Federal do ABC),Pública,2018-07-01,UFABC
374167,394930,Pelotas,Brazil,Windows,Chrome 89,Web,Windows,UFPEL (Universidade Federal de Pelotas),Pública,2018-07-01,UFPEL
374168,395209,São Bernardo do Campo,Brazil,Windows,Chrome 76,Web,Windows,UFABC (Universidade Federal do ABC),,2018-07-01,UFABC


In [21]:
df_universidade = pd.DataFrame(df['Universidade'].value_counts().sort_values(ascending=False)[:11])
df_universidade.reset_index(inplace=True)
df_universidade.columns = ['Universidade', 'Vendas']
df_universidade

,Universidade,Vendas
0,UFRJ,39621
1,UNICAMP,18924
2,UFF,17746
3,USP,14463
4,Outra,12460
5,UFABC,11567
6,UTFPR,10984
7,UFSC,9905
8,UFPE,9333
9,UFMG,8377


In [22]:
fig = go.Figure(data=go.Bar(x=df_universidade['Universidade'], y=df_universidade['Vendas'], marker={'color': 'gray'}))
fig.update_layout(font=dict(color='black'), title_text='Vendas por Universidade', barmode='group')
fig.show()

#### Sazonalidade das Top10 Faculdades

In [23]:
top10 = df_universidade['Universidade'].tolist()
top10

['UFRJ',
 'UNICAMP',
 'UFF',
 'USP',
 'Outra',
 'UFABC',
 'UTFPR',
 'UFSC',
 'UFPE',
 'UFMG',
 'PUCRIO']

In [24]:
df['top10'] = df['Universidade'].map(lambda x: 'Top10' if x in top10 else None)

In [25]:
universidades = pd.pivot_table(df.loc[df['top10'] == 'Top10'], values='user_id', aggfunc='count', columns=['data', 'Universidade']).T
universidades.reset_index(inplace=True)
universidades

,data,Universidade,user_id
0,2018-01-01,Outra,59
1,2018-01-01,PUCRIO,100
2,2018-01-01,UFABC,17
3,2018-01-01,UFF,194
4,2018-01-01,UFMG,34
...,...,...,...
567,2022-04-01,UFRJ,608
568,2022-04-01,UFSC,177
569,2022-04-01,UNICAMP,442
570,2022-04-01,USP,328


In [26]:
datas = universidades['data'].unique()

fig = go.Figure(data=[
    #go.Scatter(x=vendas['data'], y=vendas['user_id'], marker={'color': 'gray'}),
    go.Scatter(name='PUCRIO', x=datas, y=universidades.loc[universidades['Universidade'] == 'PUCRIO']['user_id'].tolist(), marker={'color': 'gray'}),
    go.Scatter(name='UFABC', x=datas, y=universidades.loc[universidades['Universidade'] == 'UFABC']['user_id'].tolist(), marker={'color': 'gray'}),
    go.Scatter(name='UFF', x=datas, y=universidades.loc[universidades['Universidade'] == 'UFF']['user_id'].tolist(), marker={'color': 'gray'}),
    go.Scatter(name='UFMG', x=datas, y=universidades.loc[universidades['Universidade'] == 'UFMG']['user_id'].tolist(), marker={'color': 'gray'}),
    go.Scatter(name='UFPE', x=datas, y=universidades.loc[universidades['Universidade'] == 'UFPE']['user_id'].tolist(), marker={'color': 'gray'}),
    go.Scatter(name='UFRJ', x=datas, y=universidades.loc[universidades['Universidade'] == 'UFRJ']['user_id'].tolist(), marker={'color': 'blue'}),
    go.Scatter(name='UFSC', x=datas, y=universidades.loc[universidades['Universidade'] == 'UFSC']['user_id'].tolist(), marker={'color': 'gray'}),
    go.Scatter(name='UNICAMP', x=datas, y=universidades.loc[universidades['Universidade'] == 'UNICAMP']['user_id'].tolist(), marker={'color': 'gray'}),
    go.Scatter(name='USP', x=datas, y=universidades.loc[universidades['Universidade'] == 'USP']['user_id'].tolist(), marker={'color': 'gray'}),
    go.Scatter(name='UTFPR', x=datas, y=universidades.loc[universidades['Universidade'] == 'UTFPR']['user_id'].tolist(), marker={'color': 'gray'}),
    ])
fig.update_layout(font=dict(color='black'), title_text='Vendas por mês Top 10 Faculdades')
fig.show()

As sazonalidades são bem parecidas, seguindo mais ou menos a sazonalidade da UFRJ (maior faculdade)

#### Proporção de vendas por mês das Top10 Faculdades

In [27]:
fig = go.Figure(data=[
    go.Bar(name='PUCRIO', x=datas, y=universidades.loc[universidades['Universidade'] == 'PUCRIO']['user_id'].tolist()),
    go.Bar(name='UFABC', x=datas, y=universidades.loc[universidades['Universidade'] == 'UFABC']['user_id'].tolist()),
    go.Bar(name='UFF', x=datas, y=universidades.loc[universidades['Universidade'] == 'UFF']['user_id'].tolist()),
    go.Bar(name='UFMG', x=datas, y=universidades.loc[universidades['Universidade'] == 'UFMG']['user_id'].tolist()),
    go.Bar(name='UFPE', x=datas, y=universidades.loc[universidades['Universidade'] == 'UFPE']['user_id'].tolist()),
    go.Bar(name='UFRJ', x=datas, y=universidades.loc[universidades['Universidade'] == 'UFRJ']['user_id'].tolist()),
    go.Bar(name='UFSC', x=datas, y=universidades.loc[universidades['Universidade'] == 'UFSC']['user_id'].tolist()),
    go.Bar(name='UNICAMP', x=datas, y=universidades.loc[universidades['Universidade'] == 'UNICAMP']['user_id'].tolist()),
    go.Bar(name='USP', x=datas, y=universidades.loc[universidades['Universidade'] == 'USP']['user_id'].tolist()),
    go.Bar(name='UTFPR', x=datas, y=universidades.loc[universidades['Universidade'] == 'UTFPR']['user_id'].tolist()),
])

fig.update_layout(font=dict(color='black'), barmode='stack', title_text='Proporção das Vendas nas Top 10 Universidades')
fig.show()

### Tipo de Universidade

In [28]:
df['gp:Tipo de Universidade'].value_counts()

Pública      300987
Privada       55327
              17843
Em branco        13
Name: gp:Tipo de Universidade, dtype: int64

In [29]:
df_tipo_universidade = pd.DataFrame(df['gp:Tipo de Universidade'].value_counts().sort_values(ascending=False))
df_tipo_universidade.reset_index(inplace=True)
df_tipo_universidade.columns = ['Tipo de Universidade', 'Vendas']
df_tipo_universidade

,Tipo de Universidade,Vendas
0,Pública,300987
1,Privada,55327
2,,17843
3,Em branco,13


In [30]:
fig = go.Figure(data=go.Bar(x=df_tipo_universidade['Tipo de Universidade'], y=df_tipo_universidade['Vendas'], marker={'color': 'gray'}))
fig.update_layout(font=dict(color='black'), title_text='Vendas por Tipo de Universidade', barmode='group')
fig.show()

In [31]:
tipo_universidade = pd.pivot_table(df, values='user_id', aggfunc='count', columns=['data', 'gp:Tipo de Universidade']).T
tipo_universidade.reset_index(inplace=True)
tipo_universidade

,data,gp:Tipo de Universidade,user_id
0,2018-01-01,,327
1,2018-01-01,Privada,132
2,2018-01-01,Pública,1360
3,2018-02-01,,327
4,2018-02-01,Privada,140
...,...,...,...
143,2022-03-01,Privada,1880
144,2022-03-01,Pública,7889
145,2022-04-01,Em branco,1
146,2022-04-01,Privada,2065


In [32]:
datas_tipo = tipo_universidade['data'].unique()

fig = go.Figure(data=[
    go.Bar(name='Pública', x=datas_tipo, y=tipo_universidade.loc[tipo_universidade['gp:Tipo de Universidade'] == 'Pública']['user_id'].tolist()),
    go.Bar(name='Privada', x=datas_tipo, y=tipo_universidade.loc[tipo_universidade['gp:Tipo de Universidade'] == 'Privada']['user_id'].tolist()),
])

fig.update_layout(font=dict(color='black'), barmode='stack', title_text='Proporção das Vendas por Tipo de Universidade')
fig.show()

## Time Series

In [33]:
vendas.head()

,data,user_id
0,2018-01-01,1819
1,2018-02-01,2012
2,2018-03-01,4035
3,2018-04-01,4305
4,2018-05-01,3956


In [34]:
vendas.set_index('data', inplace=True)

In [35]:
decomposicao = seasonal_decompose(vendas)
tendencia = decomposicao.trend
sazonal = decomposicao.seasonal
aleatorio = decomposicao.resid

In [36]:
fig = go.Figure(data=go.Scatter(x=tendencia.index, y=tendencia.values, marker={'color': 'gray'}))
fig.update_layout(font=dict(color='black'), title_text='Tendência')
fig.show()

In [37]:
fig = go.Figure(data=go.Scatter(x=sazonal.index, y=sazonal.values, marker={'color': 'gray'}))
fig.update_layout(font=dict(color='black'), title_text='Sazonalidade')
fig.show()

In [38]:
fig = go.Figure(data=go.Scatter(x=aleatorio.index, y=aleatorio.values, marker={'color': 'gray'}))
fig.update_layout(font=dict(color='black'), title_text='Resíduo')
fig.show()

## DataSet Final

In [39]:
df.rename(columns={'gp:Tipo de Universidade': 'tipo_universidade', 'Universidade': 'universidade'}, inplace=True)
colunas = ['user_id', 'city', 'country', 'device', 'platform', 'tipo_universidade', 'data', 'universidade']
df = df[colunas]
df.head()

,user_id,city,country,device,platform,tipo_universidade,data,universidade
0,124557,São Luís,Brazil,Windows,Web,Pública,2022-02-01,UFMA
1,41262,Florianópolis,Brazil,Linux,Web,Pública,2022-02-01,UFSC
2,98102,Palmas,Brazil,Windows,Web,Pública,2022-02-01,UFT
3,138808,Teresina,Brazil,Android,Web,Pública,2022-02-01,UFPI
4,144725,São Paulo,Brazil,Apple iPhone,Web,Privada,2022-02-01,UNIVATES


# Preparação dos Dados

### Feature Faculdades Top 10

In [40]:
top10.remove('Outra')

In [41]:
df['top10'] = df['universidade'].map(lambda x: na_lista(x, top10))
df.head()

,user_id,city,country,device,platform,tipo_universidade,data,universidade,top10
0,124557,São Luís,Brazil,Windows,Web,Pública,2022-02-01,UFMA,Outra
1,41262,Florianópolis,Brazil,Linux,Web,Pública,2022-02-01,UFSC,UFSC
2,98102,Palmas,Brazil,Windows,Web,Pública,2022-02-01,UFT,Outra
3,138808,Teresina,Brazil,Android,Web,Pública,2022-02-01,UFPI,Outra
4,144725,São Paulo,Brazil,Apple iPhone,Web,Privada,2022-02-01,UNIVATES,Outra


### Feature Top 20 Cidades

In [42]:
top20_cidades = df['city'].value_counts()[:20].index.tolist()

In [43]:
df['top20_cidades'] = df['city'].map(lambda x: na_lista(x, top20_cidades))
df.head()

,user_id,city,country,device,platform,tipo_universidade,data,universidade,top10,top20_cidades
0,124557,São Luís,Brazil,Windows,Web,Pública,2022-02-01,UFMA,Outra,Outra
1,41262,Florianópolis,Brazil,Linux,Web,Pública,2022-02-01,UFSC,UFSC,Florianópolis
2,98102,Palmas,Brazil,Windows,Web,Pública,2022-02-01,UFT,Outra,Outra
3,138808,Teresina,Brazil,Android,Web,Pública,2022-02-01,UFPI,Outra,Outra
4,144725,São Paulo,Brazil,Apple iPhone,Web,Privada,2022-02-01,UNIVATES,Outra,São Paulo


### Valores Nulos - Tipo de Universidade

In [44]:
df['tipo_universidade'].value_counts()

Pública      300987
Privada       55327
              17843
Em branco        13
Name: tipo_universidade, dtype: int64

In [45]:
df['tipo_universidade'] = df['tipo_universidade'].map(lambda x: trocar_valor(x, '', 'Em branco'))

In [46]:
df['tipo_universidade'].value_counts()

Pública      300987
Privada       55327
Em branco     17856
Name: tipo_universidade, dtype: int64

### Dummies

In [47]:
df_dummies = pd.get_dummies(df, columns=['top10', 'top20_cidades'])
df_dummies.head()

,user_id,city,country,device,platform,tipo_universidade,data,universidade,top10_Outra,top10_PUCRIO,...,top20_cidades_Natal,top20_cidades_Niterói,top20_cidades_Outra,top20_cidades_Porto Alegre,top20_cidades_Recife,top20_cidades_Rio de Janeiro,top20_cidades_Salvador,top20_cidades_Santo André,top20_cidades_São Carlos,top20_cidades_São Paulo
0,124557,São Luís,Brazil,Windows,Web,Pública,2022-02-01,UFMA,1,0,...,0,0,1,0,0,0,0,0,0,0
1,41262,Florianópolis,Brazil,Linux,Web,Pública,2022-02-01,UFSC,0,0,...,0,0,0,0,0,0,0,0,0,0
2,98102,Palmas,Brazil,Windows,Web,Pública,2022-02-01,UFT,1,0,...,0,0,1,0,0,0,0,0,0,0
3,138808,Teresina,Brazil,Android,Web,Pública,2022-02-01,UFPI,1,0,...,0,0,1,0,0,0,0,0,0,0
4,144725,São Paulo,Brazil,Apple iPhone,Web,Privada,2022-02-01,UNIVATES,1,0,...,0,0,0,0,0,0,0,0,0,1


In [48]:
df_dummies.columns

Index(['user_id', 'city', 'country', 'device', 'platform', 'tipo_universidade',
       'data', 'universidade', 'top10_Outra', 'top10_PUCRIO', 'top10_UFABC',
       'top10_UFF', 'top10_UFMG', 'top10_UFPE', 'top10_UFRJ', 'top10_UFSC',
       'top10_UNICAMP', 'top10_USP', 'top10_UTFPR',
       'top20_cidades_Belo Horizonte', 'top20_cidades_Brasília',
       'top20_cidades_Campinas', 'top20_cidades_Campo Grande',
       'top20_cidades_Curitiba', 'top20_cidades_Florianópolis',
       'top20_cidades_Fortaleza', 'top20_cidades_Goiânia',
       'top20_cidades_Joinville', 'top20_cidades_Juiz de Fora',
       'top20_cidades_Manaus', 'top20_cidades_Natal', 'top20_cidades_Niterói',
       'top20_cidades_Outra', 'top20_cidades_Porto Alegre',
       'top20_cidades_Recife', 'top20_cidades_Rio de Janeiro',
       'top20_cidades_Salvador', 'top20_cidades_Santo André',
       'top20_cidades_São Carlos', 'top20_cidades_São Paulo'],
      dtype='object')

### Agrupamento de Dados

In [49]:
time_series = pd.pivot_table(df, values='user_id', aggfunc='count', columns='data').T
time_series.reset_index(inplace=True)
time_series.rename(columns={'user_id':'vendas'}, inplace=True)
time_series['vendas'] = time_series['vendas'].astype('float')
time_series.head()

,data,vendas
0,2018-01-01,1819.0
1,2018-02-01,2012.0
2,2018-03-01,4035.0
3,2018-04-01,4305.0
4,2018-05-01,3956.0


In [50]:
top10_faculdades = pd.pivot_table(df_dummies, values=['top10_Outra', 'top10_PUCRIO', 'top10_UFABC',
       'top10_UFF', 'top10_UFMG', 'top10_UFPE', 'top10_UFRJ', 'top10_UFSC',
       'top10_UNICAMP', 'top10_USP', 'top10_UTFPR'], aggfunc='sum', columns='data').T
top10_faculdades.reset_index(inplace=True)
top10_faculdades.rename(columns={
    'top10_Outra': 'vendas_Outra_Faculdade', 
    'top10_PUCRIO': 'vendas_PUCRIO', 
    'top10_UFABC': 'vendas_UFABC',
    'top10_UFF': 'vendas_UFF', 
    'top10_UFMG': 'vendas_UFMG', 
    'top10_UFPE': 'vendas_UFPE', 
    'top10_UFRJ': 'vendas_UFRJ', 
    'top10_UFSC': 'vendas_UFSC',
    'top10_UNICAMP': 'vendas_UNICAMP', 
    'top10_USP': 'vendas_USP', 
    'top10_UTFPR': 'vendas_UTFPR'}, inplace=True)
top10_faculdades.head()

,data,vendas_Outra_Faculdade,vendas_PUCRIO,vendas_UFABC,vendas_UFF,vendas_UFMG,vendas_UFPE,vendas_UFRJ,vendas_UFSC,vendas_UNICAMP,vendas_USP,vendas_UTFPR
0,2018-01-01,618.0,100.0,17.0,194.0,34.0,71.0,407.0,23.0,200.0,145.0,10.0
1,2018-02-01,652.0,98.0,22.0,199.0,34.0,85.0,492.0,28.0,219.0,168.0,15.0
2,2018-03-01,926.0,229.0,16.0,357.0,46.0,163.0,1473.0,30.0,577.0,196.0,22.0
3,2018-04-01,1204.0,189.0,186.0,512.0,96.0,134.0,1098.0,151.0,434.0,218.0,83.0
4,2018-05-01,1462.0,176.0,130.0,349.0,212.0,158.0,721.0,122.0,376.0,178.0,72.0


In [51]:
top20_cidades = pd.pivot_table(df_dummies, values=['top20_cidades_Belo Horizonte', 'top20_cidades_Brasília',
       'top20_cidades_Campinas', 'top20_cidades_Campo Grande',
       'top20_cidades_Curitiba', 'top20_cidades_Florianópolis',
       'top20_cidades_Fortaleza', 'top20_cidades_Goiânia',
       'top20_cidades_Joinville', 'top20_cidades_Juiz de Fora',
       'top20_cidades_Manaus', 'top20_cidades_Natal', 'top20_cidades_Niterói',
       'top20_cidades_Outra', 'top20_cidades_Porto Alegre',
       'top20_cidades_Recife', 'top20_cidades_Rio de Janeiro',
       'top20_cidades_Salvador', 'top20_cidades_Santo André',
       'top20_cidades_São Carlos', 'top20_cidades_São Paulo'], aggfunc='sum', columns='data').T
top20_cidades.reset_index(inplace=True)
top20_cidades.rename(columns={
    'top20_cidades_Belo Horizonte': 'vendas_Belo_Horizonte', 
    'top20_cidades_Brasília': 'vendas_Brasilia',
    'top20_cidades_Campinas': 'vendas_Campinas',
    'top20_cidades_Campo Grande': 'vendas_Campo_Grande',
    'top20_cidades_Curitiba': 'vendas_Curitiba',
    'top20_cidades_Florianópolis': 'vendas_Florianopolis',
    'top20_cidades_Fortaleza': 'vendas_Fortaleza',
    'top20_cidades_Goiânia': 'vendas_Goiania',
    'top20_cidades_Joinville': 'vendas_Joinville',
    'top20_cidades_Juiz de Fora': 'vendas_Juiz_de_Fora',
    'top20_cidades_Manaus': 'vendas_Manaus',
    'top20_cidades_Natal': 'vendas_Natal',
    'top20_cidades_Niterói': 'vendas_Niteroi',
    'top20_cidades_Outra': 'vendas_Outra_Cidade',
    'top20_cidades_Porto Alegre': 'vendas_Porto_Alegre',
    'top20_cidades_Recife': 'vendas_Recife',
    'top20_cidades_Rio de Janeiro': 'vendas_Rio_de_Janeiro',
    'top20_cidades_Salvador': 'vendas_Salvador',
    'top20_cidades_Santo André': 'vendas_Santo_Andre',
    'top20_cidades_São Carlos': 'vendas_Sao_Carlos', 
    'top20_cidades_São Paulo': 'vendas_Sao_Paulo'}, inplace=True)
top20_cidades.head()

,data,vendas_Belo_Horizonte,vendas_Brasilia,vendas_Campinas,vendas_Campo_Grande,vendas_Curitiba,vendas_Florianopolis,vendas_Fortaleza,vendas_Goiania,vendas_Joinville,...,vendas_Natal,vendas_Niteroi,vendas_Outra_Cidade,vendas_Porto_Alegre,vendas_Recife,vendas_Rio_de_Janeiro,vendas_Salvador,vendas_Santo_Andre,vendas_Sao_Carlos,vendas_Sao_Paulo
0,2018-01-01,56.0,41.0,115.0,14.0,17.0,14.0,12.0,12.0,5.0,...,3.0,79.0,587.0,11.0,51.0,521.0,25.0,9.0,14.0,187.0
1,2018-02-01,56.0,38.0,121.0,18.0,22.0,21.0,13.0,11.0,7.0,...,2.0,81.0,654.0,11.0,62.0,592.0,23.0,13.0,14.0,205.0
2,2018-03-01,111.0,46.0,316.0,31.0,31.0,18.0,15.0,20.0,8.0,...,9.0,171.0,1103.0,14.0,123.0,1544.0,22.0,12.0,29.0,282.0
3,2018-04-01,137.0,80.0,233.0,26.0,114.0,95.0,18.0,29.0,17.0,...,4.0,232.0,1299.0,21.0,105.0,1300.0,38.0,40.0,27.0,359.0
4,2018-05-01,225.0,146.0,216.0,37.0,94.0,65.0,18.0,28.0,22.0,...,8.0,155.0,1250.0,20.0,110.0,1065.0,41.0,39.0,31.0,300.0


In [52]:
tendencia = pd.DataFrame(tendencia)
tendencia.reset_index(inplace=True)
tendencia.head(8)

,data,trend
0,2018-01-01,NaN
1,2018-02-01,NaN
2,2018-03-01,NaN
3,2018-04-01,NaN
4,2018-05-01,NaN
5,2018-06-01,NaN
6,2018-07-01,3863.541667
7,2018-08-01,3946.666667


In [53]:
sazonal = pd.DataFrame(sazonal)
sazonal.reset_index(inplace=True)
sazonal.head()

,data,seasonal
0,2018-01-01,-2401.784433
1,2018-02-01,-1835.228877
2,2018-03-01,935.868345
3,2018-04-01,670.798900
4,2018-05-01,267.660012


In [54]:
df_final = (time_series
            .merge(top10_faculdades, how='left', left_on='data', right_on='data')
            .merge(top20_cidades, how='left', left_on='data', right_on='data')
            .merge(tendencia, how='left', left_on='data', right_on='data')
            .merge(sazonal, how='left', left_on='data', right_on='data')
            )
df_final.head()

,data,vendas,vendas_Outra_Faculdade,vendas_PUCRIO,vendas_UFABC,vendas_UFF,vendas_UFMG,vendas_UFPE,vendas_UFRJ,vendas_UFSC,...,vendas_Outra_Cidade,vendas_Porto_Alegre,vendas_Recife,vendas_Rio_de_Janeiro,vendas_Salvador,vendas_Santo_Andre,vendas_Sao_Carlos,vendas_Sao_Paulo,trend,seasonal
0,2018-01-01,1819.0,618.0,100.0,17.0,194.0,34.0,71.0,407.0,23.0,...,587.0,11.0,51.0,521.0,25.0,9.0,14.0,187.0,NaN,-2401.784433
1,2018-02-01,2012.0,652.0,98.0,22.0,199.0,34.0,85.0,492.0,28.0,...,654.0,11.0,62.0,592.0,23.0,13.0,14.0,205.0,NaN,-1835.228877
2,2018-03-01,4035.0,926.0,229.0,16.0,357.0,46.0,163.0,1473.0,30.0,...,1103.0,14.0,123.0,1544.0,22.0,12.0,29.0,282.0,NaN,935.868345
3,2018-04-01,4305.0,1204.0,189.0,186.0,512.0,96.0,134.0,1098.0,151.0,...,1299.0,21.0,105.0,1300.0,38.0,40.0,27.0,359.0,NaN,670.798900
4,2018-05-01,3956.0,1462.0,176.0,130.0,349.0,212.0,158.0,721.0,122.0,...,1250.0,20.0,110.0,1065.0,41.0,39.0,31.0,300.0,NaN,267.660012


In [55]:
df_final.to_csv('../Dados/df_final.csv', index=None)